# synApps *busy* record

From *APS Python Training for Bluesky Data Acquisition*.

**Objective**

In this notebook, we show how to use the EPICS *busy* record.

The synApps [*busy* record](https://epics.anl.gov/bcda/synApps/busy/busyRecord.html)
is used to signal the completion of an operation.  Generally, the *busy* record is used
for operations that have no inherent means of reporting that a long or complex operation
has completed.  Two cases come to mind immediately, both involving waiting for
completion of some operation):

- arbitrary operation
- movement of a positioner (or set of positioners such as a diffractometer or hexapod)

One type of positioner, the EPICS *motor* record, already has such a means
to report done moving, via the `.DMOV` field, so the *busy* record provides
no additional benefit.  But a set of simpler PVs (using *ao*, *ai*, *bo*, & *bi*
records), which together implement the main features of a positioner, would benefit
from having a _done moving_ signal.  This is a case for use of a *busy* record.

This notebook expects an EPICS IOC with prefix `gp:` that provides several PVs:

PV | record type | description
--- | --- | ---
`{IOC}gp:bit1` | bo | general purpose binary output (bit) variable
`{IOC}gp:float1` | ao | general purpose analog output (floating-point) variable
`{IOC}gp:float2` | ao | general purpose floating-point (floating-point) variable
`{IOC}mybusy1` | busy | general purpose busy record


The `instrument` package is not necessary.  This notebook will use a [temporary databroker catalog](https://blueskyproject.io/databroker/generated/databroker.temp.html?highlight=temp#databroker.temp).

In [1]:
from apstools.synApps import BusyStatus
from apstools.utils import run_in_thread
from bluesky import plans as bp
from bluesky import plan_stubs as bps
from bluesky.callbacks.best_effort import BestEffortCallback
from ophyd import Component
from ophyd import Device
from ophyd import EpicsSignal
from ophyd import PVPositioner
from ophyd import Signal
from ophyd.status import DeviceStatus

import bluesky
import databroker
import logging
import time

cat = databroker.temp()
IOC = "gp:"
logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
RE = bluesky.RunEngine({})
RE.subscribe(cat.v1.insert)
RE.subscribe(BestEffortCallback())

1

## arbitrary operation

Use a *busy* record to indicate the state of some arbitrary operation.  A *fly* scan is one example.  In this example, a user-adjustable time delay is sued to simulate the arbitrary operation.

The *apstools* package provides support for the *busy* record:


- `apstools.synApps.BusyRecord` - ophyd device (won't need that here)
- `apstools.synApps.BusyStatus` - static values: `BusyStatus.busy` and `BusyStatus.done`

### _operation_ uses `trigger()` method

Let's start with a Device with a *busy* PV and a user-settable delay time.  The _operation_ is run (in an external thread) from the device's `.trigger()` method.

Follow example from https://blueskyproject.io/ophyd/explanations/status.html?highlight=devicestatus

In [2]:
class OperatorBase(Device):
    busy = Component(EpicsSignal, "mybusy1", kind="omitted", string=True)
    delay_time_s = Component(Signal, value=2, kind="hinted")

    def trigger(self):
        def check_busy(*, old_value, value, **kwargs):
            "Mark as finished when *busy* changes from Busy to Done."
            if old_value == BusyStatus.busy and value == BusyStatus.done:
                self.busy.clear_sub(check_busy)
                status.set_finished()
        
        @run_in_thread
        def simulated_operation():
            # simulate how the external process works
            self.busy.set(BusyStatus.busy)
            time.sleep(self.delay_time_s.get())
            self.busy.set(BusyStatus.done)

        status = DeviceStatus(self.busy)
        self.busy.subscribe(check_busy)
        simulated_operation()
        return status

In [3]:
operator = OperatorBase(IOC, name="operator")
operator.wait_for_connection()
operator.stage_sigs["delay_time_s"] = 1.0
operator.read()

OrderedDict([('operator_delay_time_s',
              {'value': 2, 'timestamp': 1639693816.0613751})])

Run the operation by calling the `operation.trigger()` method.  Since that method returns a status object (used by the RunEngine to wait for the trigger method to complete), grab that status object.  Use that to wait for the trigger method to complete.  Report elapsed time, as well.

In [4]:
t0 = time.time()  # time the trigger()
st = operator.trigger()  # trigger() returns a status object
print(f"{time.time()-t0:.3f} {st = }")
st.wait()
print(f"{time.time()-t0:.3f} {st = }")  # default time, since device was not staged

0.005 st = DeviceStatus(device=operator_busy, done=False, success=False)
2.011 st = DeviceStatus(device=operator_busy, done=True, success=True)


In the above, the elapsed time matched the default `Operator.delay_time_s` value since the device was not [_staged_](https://blueskyproject.io/ophyd/explanations/staging.html?highlight=stage) before calling the trigger method.

Repeat the same steps, adding calls to  `stage()` (before triggering) and `unstage()` (after).

In [5]:
print("with staging")
t0 = time.time()
operator.stage()
st = operator.trigger()
print(f"{time.time()-t0:.3f} {st = }")
st.wait()
operator.unstage()
print(f"{time.time()-t0:.3f} {st = }")  # staged time

with staging
0.006 st = DeviceStatus(device=operator_busy, done=False, success=False)
1.014 st = DeviceStatus(device=operator_busy, done=True, success=True)


Now, run `count()` (one of the bluesky.plans) with the `operator` device as a "detector".  The standard plans take care of staging, triggering, reading, and unstaging the device.  A RunEngine subscription by the `BestEffortCallback` is responsible for generating the `LiveTable` view.

In [6]:
RE(bp.count([operator]))



Transient Scan ID: 1     Time: 2021-12-16 16:30:19
Persistent Unique Scan ID: 'eaae0395-4c71-4339-b572-bf4e0cdd19e0'
New stream: 'primary'
+-----------+------------+-----------------------+
|   seq_num |       time | operator_delay_time_s |
+-----------+------------+-----------------------+
|         1 | 16:30:20.2 |                 1.000 |
+-----------+------------+-----------------------+
generator count ['eaae0395'] (scan num: 1)





('eaae0395-4c71-4339-b572-bf4e0cdd19e0',)

## positioner movement

Use a *busy* record to signal _done moving_ for a positioner built
from separate PVs (using `ophyd.PVPositioner`).

PV | PVPositioner attribute
--- | ---
`{IOC}gp:bit1` | `stop_signal`
`{IOC}gp:float1` | `setpoint`
`{IOC}gp:float2` | `readback`
`{IOC}gp:float2.PREC` | `precision`
`{IOC}mybusy1` | `done`

TODO: Discuss the implementation

In [7]:
class Mover(PVPositioner):
    setpoint = Component(EpicsSignal, "gp:float1")
    readback = Component(EpicsSignal, "gp:float2")
    done = Component(EpicsSignal, "mybusy1")
    done_value = 0
    stop_signal = Component(EpicsSignal, "gp:bit1")
    stop_value = 1
    precision = Component(EpicsSignal, "gp:float2.PREC")

    simulator_sleep_s = 0.1
    tolerance = 0.001

    @property
    def in_position(self):
        return abs(self.setpoint.get() - self.readback.get()) <= self.tolerance

    @property
    def is_done(self):
        return self.done.get() == self.done_value
    
    @run_in_thread
    def setpoint_watch(self, *args, **kwargs):
        if self.is_done:
            self.done.put(1 - self.done_value)

    @run_in_thread
    def motion_simulator(self):
        """Simulate the motion using Python code."""
        reset_stop_value = 1 - self.stop_value

        while True:
            if self.in_position and not self.is_done:
                # finish the move to the exact setpoint value
                self.readback.put(self.setpoint.get())
                self.done.put(self.done_value)
                logger.info(f"simulator: {self.readback.get() = } {self.is_done = } end")

            if not self.in_position:
                if self.stop_signal.get() == self.stop_value:
                    # must STOP the move now, stay at current position
                    self.setpoint.put(self.readback.get())
                    self.stop_signal.put(reset_stop_value)
                    self.done.put(self.done_value)
                    logger.info(f"simulator: {self.readback.get() = } stopped")

                diff = self.setpoint.get() - self.readback.get()

                if abs(diff) > self.tolerance:
                    # move closer to the setpoint
                    step = diff * 0.5  # novel step size
                    value = step + self.readback.get()
                    self.readback.put(value)
                    logger.info(f"simulator: {value = } {self.is_done = }")

            time.sleep(self.simulator_sleep_s)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # update tolerance based on display precision
        self.tolerance = 10**(-self.precision.get())
        self.setpoint.subscribe(self.setpoint_watch)
        self.motion_simulator()

In [8]:
mover = Mover(IOC, name="mover")
mover.wait_for_connection()

In [9]:
if mover.position == 0:
    mover.move(1)
for i in range(3):
    st = mover.move(-mover.position)
    print(f"{i}  {mover.position = } {st.elapsed = }")

INFO:__main__:simulator: self.readback.get() = -1.0 self.is_done = True end


0  mover.position = -1.0 st.elapsed = 0.06354713439941406


INFO:__main__:simulator: self.readback.get() = -1.0 stopped
INFO:__main__:simulator: value = 0.0 self.is_done = False
INFO:__main__:simulator: value = 0.5 self.is_done = False
INFO:__main__:simulator: value = 0.75 self.is_done = False
INFO:__main__:simulator: value = 0.875 self.is_done = False
INFO:__main__:simulator: value = 0.9375 self.is_done = False
INFO:__main__:simulator: value = 0.96875 self.is_done = False
INFO:__main__:simulator: value = 0.984375 self.is_done = False
INFO:__main__:simulator: value = 0.9921875 self.is_done = False
INFO:__main__:simulator: value = 0.99609375 self.is_done = False
INFO:__main__:simulator: value = 0.998046875 self.is_done = False
INFO:__main__:simulator: value = 0.9990234375 self.is_done = False
INFO:__main__:simulator: value = 0.99951171875 self.is_done = False
INFO:__main__:simulator: value = 0.999755859375 self.is_done = False
INFO:__main__:simulator: value = 0.9998779296875 self.is_done = False
INFO:__main__:simulator: value = 0.99993896484375 

1  mover.position = 1.0 st.elapsed = 1.6325547695159912


INFO:__main__:simulator: self.readback.get() = 1.0 self.is_done = True end
INFO:__main__:simulator: value = 0.0 self.is_done = False
INFO:__main__:simulator: value = -0.5 self.is_done = False
INFO:__main__:simulator: value = -0.75 self.is_done = False
INFO:__main__:simulator: value = -0.875 self.is_done = False
INFO:__main__:simulator: value = -0.9375 self.is_done = False
INFO:__main__:simulator: value = -0.96875 self.is_done = False
INFO:__main__:simulator: value = -0.984375 self.is_done = False
INFO:__main__:simulator: value = -0.9921875 self.is_done = False
INFO:__main__:simulator: value = -0.99609375 self.is_done = False
INFO:__main__:simulator: value = -0.998046875 self.is_done = False
INFO:__main__:simulator: value = -0.9990234375 self.is_done = False
INFO:__main__:simulator: value = -0.99951171875 self.is_done = False
INFO:__main__:simulator: value = -0.999755859375 self.is_done = False
INFO:__main__:simulator: value = -0.9998779296875 self.is_done = False
INFO:__main__:simulato

2  mover.position = -1.0 st.elapsed = 1.6339402198791504

INFO:__main__:simulator: self.readback.get() = -1.0 self.is_done = True end


In [10]:
def n_moves(n=2):
    for _ in range(n):
        yield from bps.mv(mover, -mover.position)

RE(n_moves())

INFO:__main__:simulator: value = 0.0 self.is_done = False
INFO:__main__:simulator: value = 0.5 self.is_done = False
INFO:__main__:simulator: value = 0.75 self.is_done = False
INFO:__main__:simulator: value = 0.875 self.is_done = False
INFO:__main__:simulator: value = 0.9375 self.is_done = False
INFO:__main__:simulator: value = 0.96875 self.is_done = False
INFO:__main__:simulator: value = 0.984375 self.is_done = False
INFO:__main__:simulator: value = 0.9921875 self.is_done = False
INFO:__main__:simulator: value = 0.99609375 self.is_done = False
INFO:__main__:simulator: value = 0.998046875 self.is_done = False
INFO:__main__:simulator: value = 0.9990234375 self.is_done = False
INFO:__main__:simulator: value = 0.99951171875 self.is_done = False
INFO:__main__:simulator: value = 0.999755859375 self.is_done = False
INFO:__main__:simulator: value = 0.9998779296875 self.is_done = False
INFO:__main__:simulator: value = 0.99993896484375 self.is_done = False
INFO:__main__:simulator: self.readback.

()